<a href="https://colab.research.google.com/github/AllanKamimura/Pai_ta_ON_python/blob/main/2021_SME0340_ODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
#@title ## Concentração { run: "auto", form-width: "20%", display-mode: "form" }
#@markdown $A \underset{k_{-1}}{\stackrel{k_1}{\rightleftharpoons}}{B}{\stackrel{k_2}{\rightarrow}} C$
#@markdown ---
#@markdown ### Escolha o Método
autovetor = True #@param {type:"boolean"}
laplace = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Selecione a Aproximação
#@markdown * **Estado Quase Estacionario:** $k_2$ >> $k_{-1}$ > $k_1$ \
#@markdown * **Aproximacao Pre-Equilibrio:** $k_1$ > $k_{-1}$ >> $k_2$ \
#@markdown * **$k_{-1}$ proximo de 0:** $k_1$ $\approx k_2$ >> $k_{-1}$ \

options = "Aproximacao Pre-Equilibrio" #@param ["Estado Quase Estacionario", "Aproximacao Pre-Equilibrio", "k_-1 proximo de 0"]
#@markdown ---
#@markdown ### Resolução do Gráfico
_range = 5 #@param {type:"slider", min:0.05, max:10, step:0.05}

if options == "Estado Quase Estacionario":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 1
    km1 = 10
    k2 = 1000

if options == "Aproximacao Pre-Equilibrio":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 200
    km1 = 100
    k2 = 1

if options == "k_-1 proximo de 0":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 1
    km1 = 0.0001
    k2 = 1

import sympy

t = sympy.symbols("t")

lamb1 = sympy.symbols("lambda1")
lamb2 = sympy.symbols("lambda2")
lamb3 = sympy.symbols("lambda3")

A = sympy.Function("A")
B = sympy.Function("B")
C = sympy.Function("C")

if laplace | autovetor:
    import plotly.graph_objects as go

    fig = go.Figure(layout_title_text="Gráfico {}: Concetração pelo tempo".format(options))

    if laplace:
        deltaf = 4 * k1 * k2 - (k1 + k2 + km1) ** 2
        kf = (k1 + k2 + km1)
        theta = sympy.sqrt(-deltaf) * t / 2
        phi = - kf * t / 2

        A_dummy = A0 * sympy.exp(phi) * (sympy.cosh(theta) - (-k2-km1+k1) / sympy.sqrt(-deltaf) * sympy.sinh(theta)) + B0 * sympy.exp(phi) * (2 * km1 / sympy.sqrt(-deltaf) * sympy.sinh(theta))
        B_dummy = A0 * sympy.exp(phi) * 2 * k1 / sympy.sqrt(-deltaf) * sympy.sinh(theta) + B0 * sympy.exp(phi) * (sympy.sinh(theta) * (-km1 + k1 - k2) / sympy.sqrt(-deltaf) + sympy.cosh(theta))
        C_dummy = A0 * (1 - sympy.exp(phi) * 
                        (kf / sympy.sqrt(-deltaf) * sympy.sinh(theta) + sympy.cosh(theta))) \
                        + B0 * (1 - \
                                sympy.exp(phi) / (2 * k1) * \
                                (sympy.sinh(theta) * (sympy.sqrt(-deltaf) + kf * (-km1 + k1 - k2) / sympy.sqrt(-deltaf)) \
                                + 2 * k1 * sympy.cosh(theta))) + C0

        xl = sympy.Matrix(
            [sympy.Equality(A(t), A_dummy, evaluate = False),
                sympy.Equality(B(t), B_dummy, evaluate = False),
                sympy.Equality(C(t), C_dummy, evaluate = False),
                ]
        )

        import numpy as np

        def transform(t_eval):
            return np.array([xl[0].rhs.subs(t, t_eval).evalf(),
                            xl[1].rhs.subs(t, t_eval).evalf(),
                            xl[2].rhs.subs(t, t_eval).evalf()])

        import pandas as pd

        df2 = pd.DataFrame(
            map(transform, np.linspace(0.0001,_range,50)),
            columns = ["A(t)", "B(t)", "C(t)"],
            dtype = "float64"
            )
        
        fig.add_trace(go.Scatter(y = df2["A(t)"],
                    mode = "lines+markers", 
                    name = "A(t)_Laplace"))

        fig.add_trace(go.Scatter(y = df2["B(t)"],
                            mode = "lines+markers", 
                            name = "B(t)_Laplace"))

        fig.add_trace(go.Scatter(y = df2["C(t)"],
                            mode = "lines+markers", 
                            name = "C(t)_Laplace"))

    if autovetor:
        X = sympy.Matrix([A(t),B(t),C(t)])
        dX = sympy.diff(X, t)

        M = sympy.Matrix([[-k1, km1, 0], [k1, -k2-km1, 0], [0, k2, 0]])

        xh = sympy.solvers.ode.dsolve(dX - M*X, ics = {A(0) : A0, B(0) : B0, C(0) : C0})

        import numpy as np

        def matrix(t_eval):
            return np.array([sympy.lambdify(t, xh[0].rhs, "numpy")(t_eval),
                            sympy.lambdify(t, xh[1].rhs, "numpy")(t_eval),
                            sympy.lambdify(t, xh[2].rhs, "numpy")(t_eval)])

        import pandas as pd

        df = pd.DataFrame(
            map(matrix, np.linspace(0.0001,_range,50)),
            columns = ["A(t)", "B(t)", "C(t)"]
            )
        
        fig.add_trace(go.Scatter(y = df["A(t)"],
                            mode = "lines+markers", 
                            name = "A(t)_Autovetor"))

        fig.add_trace(go.Scatter(y = df["B(t)"],
                            mode = "lines+markers", 
                            name = "B(t)_Autovetor"))

        fig.add_trace(go.Scatter(y = df["C(t)"],
                            mode = "lines+markers", 
                            name = "C(t)_Autovetor"))
        
    fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
    fig.update_yaxes(title_text = "Concentração (unidades relativas)")
    fig.show()

    import IPython
    if laplace:
        print("Equações Laplace")
        IPython.display.display_latex(xl)
        print("\n")

    if autovetor:
        print("Equações Autovetores")
        new_xh = sympy.Matrix(xh)
        IPython.display.display_latex(new_xh)
        print("\n")
    

Equações Laplace




Equações Autovetores


In [30]:
sympy.latex(xl)

'\\left[\\begin{matrix}A{\\left(t \\right)} = \\left(- \\frac{99 \\sqrt{89801} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{89801} + \\cosh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}\\right) e^{- \\frac{301 t}{2}}\\\\B{\\left(t \\right)} = \\frac{400 \\sqrt{89801} e^{- \\frac{301 t}{2}} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{89801}\\\\C{\\left(t \\right)} = - \\left(\\frac{301 \\sqrt{89801} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{89801} + \\cosh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}\\right) e^{- \\frac{301 t}{2}} + 1\\end{matrix}\\right]'

In [31]:
#@title ## Velocidade
#@markdown ---
#@markdown ### Escolha o Método
autovetor = True #@param {type:"boolean"}
laplace = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Resolução do Gráfico
_range = 5 #@param {type:"slider", min:0.05, max:10, step:0.05}

if laplace | autovetor:
    import plotly.graph_objects as go

    fig = go.Figure(layout_title_text = "Gráfico {}: Velocidade pelo tempo".format(options))

    if laplace:
        import numpy as np

        def transform(t_eval):
            return np.array([sympy.diff(xl[0].rhs, t).subs(t, t_eval).evalf(),
                            sympy.diff(xl[1].rhs, t).subs(t, t_eval).evalf(),
                            sympy.diff(xl[2].rhs, t).subs(t, t_eval).evalf()])

        import pandas as pd

        ddf2 = pd.DataFrame(
            map(transform, np.linspace(0.05,_range,50)),
            columns = ["dA(t)/dt", "dB(t)/dt", "dC(t)/dt"],
            dtype = "float64"
            )
        
        fig.add_trace(go.Scatter(y = ddf2["dA(t)/dt"],
                    mode = "lines+markers", 
                    name = "dA(t)/dt_Laplace"))

        fig.add_trace(go.Scatter(y = ddf2["dB(t)/dt"],
                            mode = "lines+markers", 
                            name = "dB(t)/dt_Laplace"))

        fig.add_trace(go.Scatter(y = ddf2["dC(t)/dt"],
                            mode = "lines+markers", 
                            name = "dC(t)/dt_Laplace"))

    if autovetor:
        import numpy as np

        def matrix(t_eval):
            return np.array([sympy.lambdify(t, sympy.diff(xh[0].rhs, t), "numpy")(t_eval),
                            sympy.lambdify(t, sympy.diff(xh[1].rhs, t), "numpy")(t_eval),
                            sympy.lambdify(t, sympy.diff(xh[2].rhs, t), "numpy")(t_eval)])

        import pandas as pd

        ddf = pd.DataFrame(
            map(matrix, np.linspace(0.05,_range,50)),
            columns = ["dA(t)/dt", "dB(t)/dt", "dC(t)/dt"]
            )
        
        fig.add_trace(go.Scatter(y = ddf["dA(t)/dt"],
                            mode = "lines+markers", 
                            name = "dA(t)/dt_Autovetor"))

        fig.add_trace(go.Scatter(y = ddf["dB(t)/dt"],
                            mode = "lines+markers", 
                            name = "dB(t)/dt_Autovetor"))

        fig.add_trace(go.Scatter(y = ddf["dC(t)/dt"],
                            mode = "lines+markers", 
                            name = "dC(t)/dt_Autovetor"))
        
    fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
    fig.update_yaxes(title_text = "Velocidade (unidades relativas)")
    fig.show()

    import IPython
    if laplace:
        print("Equações Laplace")
        dxl = sympy.Matrix([sympy.Equality(sympy.diff(xl[0].lhs, t), sympy.diff(xl[0].rhs, t)),
                            sympy.Equality(sympy.diff(xl[1].lhs, t), sympy.diff(xl[1].rhs, t)),
                            sympy.Equality(sympy.diff(xl[2].lhs, t), sympy.diff(xl[2].rhs, t))])
        IPython.display.display_latex(dxl)
        print("\n")

    if autovetor:
        print("Equações Autovetores")
        new_xh = sympy.Matrix(xh)
        dxh = sympy.Matrix([sympy.Equality(sympy.diff(xh[0].lhs, t), sympy.diff(xh[0].rhs, t)),
                            sympy.Equality(sympy.diff(xh[1].lhs, t), sympy.diff(xh[1].rhs, t)),
                            sympy.Equality(sympy.diff(xh[2].lhs, t), sympy.diff(xh[2].rhs, t))])
        IPython.display.display_latex(dxh)
        print("\n")
    

Equações Laplace




Equações Autovetores


In [32]:
sympy.latex(dxl)

'\\left[\\begin{matrix}\\frac{d}{d t} A{\\left(t \\right)} = - \\frac{301 \\left(- \\frac{99 \\sqrt{89801} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{89801} + \\cosh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}\\right) e^{- \\frac{301 t}{2}}}{2} + \\left(\\frac{\\sqrt{89801} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{2} - \\frac{99 \\cosh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{2}\\right) e^{- \\frac{301 t}{2}}\\\\\\frac{d}{d t} B{\\left(t \\right)} = - \\frac{60200 \\sqrt{89801} e^{- \\frac{301 t}{2}} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{89801} + 200 e^{- \\frac{301 t}{2}} \\cosh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}\\\\\\frac{d}{d t} C{\\left(t \\right)} = \\left(- \\frac{\\sqrt{89801} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{2} - \\frac{301 \\cosh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{2}\\right) e^{- \\frac{301 t}{2}} - \\frac{301 \\left(- \\frac{301 \\sqrt{89801} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{89801} 

In [33]:
#@title ## Aceleração
#@markdown ---
#@markdown ### Escolha o Método
autovetor = True #@param {type:"boolean"}
laplace = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Resolução do Gráfico
_range = 5 #@param {type:"slider", min:0.05, max:10, step:0.05}

if laplace | autovetor:
    import plotly.graph_objects as go

    fig = go.Figure(layout_title_text = "Gráfico {}: Aceleração pelo tempo".format(options))

    if laplace:
        import numpy as np

        def transform(t_eval):
            return np.array([sympy.diff(xl[0].rhs, t, t).subs(t, t_eval).evalf(),
                            sympy.diff(xl[1].rhs, t, t).subs(t, t_eval).evalf(),
                            sympy.diff(xl[2].rhs, t, t).subs(t, t_eval).evalf()])

        import pandas as pd

        dddf2 = pd.DataFrame(
            map(transform, np.linspace(0.05,_range,50)),
            columns = ["d²A(t)/dt²", "d²B(t)/dt²", "d²C(t)/dt²"],
            dtype = "float64"
            )
        
        fig.add_trace(go.Scatter(y = dddf2["d²A(t)/dt²"],
                    mode = "lines+markers", 
                    name = "d²A(t)/dt²_Laplace"))

        fig.add_trace(go.Scatter(y = dddf2["d²B(t)/dt²"],
                            mode = "lines+markers", 
                            name = "d²B(t)/dt²_Laplace"))

        fig.add_trace(go.Scatter(y = dddf2["d²C(t)/dt²"],
                            mode = "lines+markers", 
                            name = "d²C(t)/dt²_Laplace"))

    if autovetor:
        import numpy as np

        def matrix(t_eval):
            return np.array([sympy.lambdify(t, sympy.diff(xh[0].rhs, t, t), "numpy")(t_eval),
                            sympy.lambdify(t, sympy.diff(xh[1].rhs, t, t), "numpy")(t_eval),
                            sympy.lambdify(t, sympy.diff(xh[2].rhs, t, t), "numpy")(t_eval)])

        import pandas as pd

        dddf = pd.DataFrame(
            map(matrix, np.linspace(0.05,_range,50)),
            columns = ["d²A(t)/dt²", "d²B(t)/dt²", "d²C(t)/dt²"]
            )
        
        fig.add_trace(go.Scatter(y = dddf["d²A(t)/dt²"],
                            mode = "lines+markers", 
                            name = "d²A(t)/dt²_Autovetor"))

        fig.add_trace(go.Scatter(y = dddf["d²B(t)/dt²"],
                            mode = "lines+markers", 
                            name = "d²B(t)/dt²_Autovetor"))

        fig.add_trace(go.Scatter(y = dddf["d²C(t)/dt²"],
                            mode = "lines+markers", 
                            name = "d²C(t)/dt²_Autovetor"))
        
    fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
    fig.update_yaxes(title_text = "Velocidade (unidades relativas)")
    fig.show()

    import IPython
    if laplace:
        print("Equações Laplace")
        ddxl = sympy.Matrix([sympy.Equality(sympy.diff(xl[0].lhs, t, t), sympy.diff(xl[0].rhs, t, t)),
                            sympy.Equality(sympy.diff(xl[1].lhs, t, t), sympy.diff(xl[1].rhs, t, t)),
                            sympy.Equality(sympy.diff(xl[2].lhs, t, t), sympy.diff(xl[2].rhs, t, t))])
        IPython.display.display_latex(ddxl)
        print("\n")

    if autovetor:
        print("Equações Autovetores")
        new_xh = sympy.Matrix(xh)
        ddxh = sympy.Matrix([sympy.Equality(sympy.diff(xh[0].lhs, t, t), sympy.diff(xh[0].rhs, t, t)),
                            sympy.Equality(sympy.diff(xh[1].lhs, t, t), sympy.diff(xh[1].rhs, t, t)),
                            sympy.Equality(sympy.diff(xh[2].lhs, t, t), sympy.diff(xh[2].rhs, t, t))])
        IPython.display.display_latex(ddxh)
        print("\n")
    

Equações Laplace




Equações Autovetores


In [34]:
sympy.latex(ddxl)

'\\left[\\begin{matrix}\\frac{d^{2}}{d t^{2}} A{\\left(t \\right)} = - \\frac{\\left(\\frac{71920000 \\sqrt{89801} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{89801} - 240000 \\cosh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}\\right) e^{- \\frac{301 t}{2}}}{4}\\\\\\frac{d^{2}}{d t^{2}} B{\\left(t \\right)} = 200 \\left(\\frac{90201 \\sqrt{89801} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}}{89801} - 301 \\cosh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}\\right) e^{- \\frac{301 t}{2}}\\\\\\frac{d^{2}}{d t^{2}} C{\\left(t \\right)} = \\frac{\\left(- 120400 \\sqrt{89801} \\sinh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)} + 35920400 \\cosh{\\left(\\frac{\\sqrt{89801} t}{2} \\right)}\\right) e^{- \\frac{301 t}{2}}}{179602}\\end{matrix}\\right]'

In [35]:
#@title ## Todos os valores { run: "auto", form-width: "20%", display-mode: "form" }

#@markdown ---
#@markdown ### Escolha o Método
metodo = "laplace" #@param ["autovetor", "laplace"]

import pandas as pd
if metodo == "autovetor":
    show = pd.concat([df,ddf,dddf], axis = 1)

if metodo == "laplace":
    show = pd.concat([df2,ddf2,dddf2], axis = 1)

show

,A(t),B(t),C(t),dA(t)/dt,dB(t)/dt,dC(t)/dt,d²A(t)/dt²,d²B(t)/dt²,d²C(t)/dt²
0,0.980297,0.019702,9.900416e-07,-0.215722,-0.429827,0.645549,0.161665,0.268162,-0.429827
1,0.312801,0.623520,6.367911e-02,-0.201631,-0.401919,0.603550,0.134271,0.267648,-0.401919
2,0.292252,0.582559,1.251888e-01,-0.188513,-0.375770,0.564283,0.125535,0.250235,-0.375770
3,0.273054,0.544289,1.826578e-01,-0.176248,-0.351323,0.527572,0.117368,0.233955,-0.351323
4,0.255116,0.508533,2.363514e-01,-0.164782,-0.328466,0.493248,0.109732,0.218734,-0.328466
5,0.238356,0.475126,2.865178e-01,-0.154061,-0.307097,0.461158,0.102593,0.204503,-0.307097
6,0.222698,0.443913,3.333886e-01,-0.144038,-0.287117,0.431155,0.095919,0.191199,-0.287117
7,0.208068,0.414751,3.771802e-01,-0.134667,-0.268438,0.403105,0.089678,0.178759,-0.268438
8,0.194400,0.387505,4.180951e-01,-0.125906,-0.250973,0.376879,0.083844,0.167129,-0.250973
9,0.181629,0.362049,4.563222e-01,-0.117715,-0.234645,0.352360,0.078389,0.156256,-0.234645


In [36]:
#@title ## Parametros da Reação e Condições iniciais { run: "auto", form-width: "20%", display-mode: "form" }
#@markdown $A \underset{k_{-1}}{\stackrel{k_1}{\rightleftharpoons}}{B}{\stackrel{k_2}{\rightarrow}} C$

#@markdown ---
#@markdown ### Escolha o Método
autovetor = True #@param {type:"boolean"}
laplace = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Condição Inicial
A_0 = 1 #@param {type:"slider", min:0.05, max:1, step:0.05}
B_0 = 0.35 #@param {type:"slider", min:0.00, max:1, step:0.05}
C_0 = 0.05 #@param {type:"slider", min:0.00, max:1, step:0.05}
#@markdown ---
#@markdown ### Constantes de Equilibrio
k1 = 1 #@param {type:"number"}
km1 = 0.001#@param {type:"number"}
k2 = 1 #@param {type:"number"}
#@markdown ---
#@markdown ### Resolução do Gráfico
_range = 5.3 #@param {type:"slider", min:0.05, max:10, step:0.05}

A0 = A_0 / (A_0 + B_0 + C_0)
B0 = B_0 / (A_0 + B_0 + C_0)
C0 = C_0 / (A_0 + B_0 + C_0)

options = "geral"

import sympy

t = sympy.symbols("t")

lamb1 = sympy.symbols("lambda1")
lamb2 = sympy.symbols("lambda2")
lamb3 = sympy.symbols("lambda3")

A = sympy.Function("A")
B = sympy.Function("B")
C = sympy.Function("C")

if laplace | autovetor:
    import plotly.graph_objects as go

    fig = go.Figure(layout_title_text="Gráfico {}: Concetração pelo tempo".format(options))

    if laplace:
        deltaf = 4 * k1 * k2 - (k1 + k2 + km1) ** 2
        kf = (k1 + k2 + km1)
        theta = sympy.sqrt(-deltaf) * t / 2
        phi = - kf * t / 2

        A_dummy = A0 * sympy.exp(phi) * (sympy.cosh(theta) - (-k2-km1+k1) / sympy.sqrt(-deltaf) * sympy.sinh(theta)) + B0 * sympy.exp(phi) * (2 * km1 / sympy.sqrt(-deltaf) * sympy.sinh(theta))
        B_dummy = A0 * sympy.exp(phi) * 2 * k1 / sympy.sqrt(-deltaf) * sympy.sinh(theta) + B0 * sympy.exp(phi) * (sympy.sinh(theta) * (-km1 + k1 - k2) / sympy.sqrt(-deltaf) + sympy.cosh(theta))
        C_dummy = A0 * (1 - sympy.exp(phi) * 
                        (kf / sympy.sqrt(-deltaf) * sympy.sinh(theta) + sympy.cosh(theta))) \
                        + B0 * (1 - \
                                sympy.exp(phi) / (2 * k1) * \
                                (sympy.sinh(theta) * (sympy.sqrt(-deltaf) + kf * (-km1 + k1 - k2) / sympy.sqrt(-deltaf)) \
                                + 2 * k1 * sympy.cosh(theta))) + C0

        xl = sympy.Matrix(
            [sympy.Equality(A(t), A_dummy, evaluate = False),
                sympy.Equality(B(t), B_dummy, evaluate = False),
                sympy.Equality(C(t), C_dummy, evaluate = False),
                ]
        )

        import numpy as np

        def transform(t_eval):
            return np.array([xl[0].rhs.subs(t, t_eval).evalf(),
                            xl[1].rhs.subs(t, t_eval).evalf(),
                            xl[2].rhs.subs(t, t_eval).evalf()])

        import pandas as pd

        df2 = pd.DataFrame(
            map(transform, np.linspace(0.0001,_range,50)),
            columns = ["A(t)", "B(t)", "C(t)"],
            dtype = "float64"
            )
        
        fig.add_trace(go.Scatter(y = df2["A(t)"],
                    mode = "lines+markers", 
                    name = "A(t)_Laplace"))

        fig.add_trace(go.Scatter(y = df2["B(t)"],
                            mode = "lines+markers", 
                            name = "B(t)_Laplace"))

        fig.add_trace(go.Scatter(y = df2["C(t)"],
                            mode = "lines+markers", 
                            name = "C(t)_Laplace"))

    if autovetor:
        X = sympy.Matrix([A(t),B(t),C(t)])
        dX = sympy.diff(X, t)

        M = sympy.Matrix([[-k1, km1, 0], [k1, -k2-km1, 0], [0, k2, 0]])

        xh = sympy.solvers.ode.dsolve(dX - M*X, ics = {A(0) : A0, B(0) : B0, C(0) : C0})

        import numpy as np

        def matrix(t_eval):
            return np.array([sympy.lambdify(t, xh[0].rhs, "numpy")(t_eval),
                            sympy.lambdify(t, xh[1].rhs, "numpy")(t_eval),
                            sympy.lambdify(t, xh[2].rhs, "numpy")(t_eval)])

        import pandas as pd

        df = pd.DataFrame(
            map(matrix, np.linspace(0.0001,_range,50)),
            columns = ["A(t)", "B(t)", "C(t)"]
            )
        
        fig.add_trace(go.Scatter(y = df["A(t)"],
                            mode = "lines+markers", 
                            name = "A(t)_Autovetor"))

        fig.add_trace(go.Scatter(y = df["B(t)"],
                            mode = "lines+markers", 
                            name = "B(t)_Autovetor"))

        fig.add_trace(go.Scatter(y = df["C(t)"],
                            mode = "lines+markers", 
                            name = "C(t)_Autovetor"))
        
    fig.update_xaxes(title_text = "tempo (unidades arbitrarias)")
    fig.update_yaxes(title_text = "Concentração (unidades relativas)")
    fig.show()

    import IPython
    if laplace:
        print("Equações Laplace")
        IPython.display.display_latex(xl)
        print("\n")

    if autovetor:
        print("Equações Autovetores")
        new_xh = sympy.Matrix(xh)
        IPython.display.display_latex(new_xh)
        print("\n")
    

Equações Laplace




Equações Autovetores
